<a href="https://colab.research.google.com/github/Julian6262/the_founder/blob/main/home%20work%209/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Навигация по уроку**

1. [Краткая история нейронных сетей. Знакомство с многослойным персептроном](https://colab.research.google.com/drive/1-FEYLgNSN5kJyAOoaW9dOhDUABU8VJlc)
2. [Математический аппарат обучения нейронных сетей](https://colab.research.google.com/drive/1FtyyBXYq9FaOEknvu4ghim-ejuO_3aIZ)
3. [Обучение многослойного персептрона в TensorFlow (Практика)](https://colab.research.google.com/drive/1Q8ioc9pkkaqGravu37C7jKw9OaG7cNL0)
4. Домашняя работа

Используя модель обучения многослойного персептрона из практической части урока (9.3), выполните следующее:

1. Увеличьте число слоев до 4-х и сравните время обучения модели и точность на тестовой выборке. За выполнение задания - 2 балла.
2. В качестве датасета использовать любой из наборов данных TensorFlow. Обучить модель. Добейтесь результата распознования более 85% на тестовой выборке. [Датасеты на выбор](https://www.tensorflow.org/datasets/overview). За задание дополнительно - 3 балла.

Для прохождения урока достаточно решить первое задание.

#####  **1. Увеличьте число слоев до 4-х и сравните время обучения модели и точность на тестовой выборке. За выполнение задания - 2 балла.**

In [ ]:
import time
import tensorflow as tf
import tensorflow_datasets as tfds

Инициализация функций/классов

In [ ]:
def xavier_init(shape):
    in_dim, out_dim = shape
    xavier_lim = tf.sqrt(6.) / tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
    weight_vals = tf.random.uniform(shape=(in_dim, out_dim), minval=-xavier_lim, maxval=xavier_lim)
    return weight_vals


def cross_entropy_loss(y_pred, y):
    sparse_ce = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_pred)
    return tf.reduce_mean(sparse_ce)


def accuracy(y_pred, y):
    class_preds = tf.argmax(tf.nn.softmax(y_pred), axis=1)
    is_equal = tf.equal(y, class_preds)
    return tf.reduce_mean(tf.cast(is_equal, tf.float32))


class DenseLayer(tf.Module):
    def __init__(self, out_dim, weight_init=xavier_init, activation=tf.identity):
        self.out_dim = out_dim
        self.weight_init = weight_init
        self.activation = activation
        self.built = False

    @tf.function
    def __call__(self, x):
        if not self.built:
            self.in_dim = x.shape[1]
            self.w = tf.Variable(self.weight_init(shape=(self.in_dim, self.out_dim)))
            self.b = tf.Variable(tf.zeros(shape=(self.out_dim,)))
            self.built = True
        y = tf.add(tf.matmul(x, self.w), self.b)
        return self.activation(y)


class MLP(tf.Module):
    def __init__(self, layers):
        self.layers = layers

    @tf.function
    def __call__(self, x, preds=False):
        for layer in self.layers:
            x = layer(x)
        return x


class Adam:
    def __init__(self, learning_rate=1e-3, beta_1=0.9, beta_2=0.999, ep=1e-7):
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.learning_rate = learning_rate
        self.ep = ep
        self.t = 1.
        self.v_dvar, self.s_dvar = [], []
        self.built = False

    def apply_gradients(self, grads, vars):
        if not self.built:
            for var in vars:
                v = tf.Variable(tf.zeros(shape=var.shape))
                s = tf.Variable(tf.zeros(shape=var.shape))
                self.v_dvar.append(v)
                self.s_dvar.append(s)
            self.built = True

        for i, (d_var, var) in enumerate(zip(grads, vars)):
            self.v_dvar[i].assign(self.beta_1 * self.v_dvar[i] + (1 - self.beta_1) * d_var)
            self.s_dvar[i].assign(self.beta_2 * self.s_dvar[i] + (1 - self.beta_2) * tf.square(d_var))
            v_dvar_bc = self.v_dvar[i] / (1 - (self.beta_1 ** self.t))
            s_dvar_bc = self.s_dvar[i] / (1 - (self.beta_2 ** self.t))
            var.assign_sub(self.learning_rate * (v_dvar_bc / (tf.sqrt(s_dvar_bc) + self.ep)))
        self.t += 1.
        return


def train_step(x_batch, y_batch, loss, acc, model, optimizer):
    with tf.GradientTape() as tape:
        y_pred = model(x_batch)
        batch_loss = loss(y_pred, y_batch)
    batch_acc = acc(y_pred, y_batch)
    grads = tape.gradient(batch_loss, model.variables)
    optimizer.apply_gradients(grads, model.variables)
    return batch_loss, batch_acc


def val_step(x_batch, y_batch, loss, acc, model):
    y_pred = model(x_batch)
    batch_loss = loss(y_pred, y_batch)
    batch_acc = acc(y_pred, y_batch)
    return batch_loss, batch_acc


def train_model(mlp, train_data, val_data, loss, acc, optimizer, epochs):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    # Отформатируем тренировочный цикл и начнем обучение
    for epoch in range(epochs):
        batch_losses_train, batch_accs_train = [], []
        batch_losses_val, batch_accs_val = [], []

        # Выполняем итерацию по обучающим данным
        for x_batch, y_batch in train_data:
            batch_loss, batch_acc = train_step(x_batch, y_batch, loss, acc, mlp, optimizer)
            batch_losses_train.append(batch_loss)
            batch_accs_train.append(batch_acc)

        # Повторяем процедуру проверки данных
        for x_batch, y_batch in val_data:
            batch_loss, batch_acc = val_step(x_batch, y_batch, loss, acc, mlp)
            batch_losses_val.append(batch_loss)
            batch_accs_val.append(batch_acc)

        # Следим за производительностью модели на уровне эпохи
        train_loss, train_acc = tf.reduce_mean(batch_losses_train), tf.reduce_mean(batch_accs_train)
        val_loss, val_acc = tf.reduce_mean(batch_losses_val), tf.reduce_mean(batch_accs_val)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        print(f"Эпоха: {epoch}")
        print(f"Обучающая ошибка: {train_loss:.3f}, Обучающая точность: {train_acc:.3f}")
        print(f"Валидационная ошибка: {val_loss:.3f}, Валидационная точность: {val_acc:.3f}")
    return train_losses, train_accs, val_losses, val_accs


def preprocess(x, y):
    x = tf.reshape(x, shape=(-1, 784))
    x = x / 255
    return x, y


class ExportModule(tf.Module):
    def __init__(self, model, preprocess, class_pred):
        self.model = model
        self.preprocess = preprocess
        self.class_pred = class_pred

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, None, None, None], dtype=tf.uint8)])
    def __call__(self, x):
        x = self.preprocess(x)
        y = self.model(x)
        y = self.class_pred(y)
        return y


def preprocess_test(x):
    x = tf.reshape(x, shape=[-1, 784])
    x = x / 255
    return x


def class_pred_test(y):
    return tf.argmax(tf.nn.softmax(y), axis=1)


def accuracy_score(y_pred, y):
    is_equal = tf.equal(y_pred, y)
    return tf.reduce_mean(tf.cast(is_equal, tf.float32))

Загрузка данных.



In [ ]:
train_data, val_data = tfds.load("mnist", split=['train[10000:]', 'train[0:10000]'], batch_size=128, as_supervised=True)
x_test, y_test = tfds.load("mnist", split=['test'], batch_size=-1, as_supervised=True)[0]

train_data, val_data = train_data.map(preprocess), val_data.map(preprocess)

hidden_layer_1_size = 700
hidden_layer_2_size = 500
hidden_layer_3_size = 300
hidden_layer_4_size = 100
output_size = 10

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


Обучение многослойного персептрона из 2х слоев

In [ ]:
print('Обучение многослойного персептрона из 2х слоев\n')
mlp_model2 = MLP([DenseLayer(out_dim=hidden_layer_1_size, activation=tf.nn.relu),
                  DenseLayer(out_dim=hidden_layer_2_size, activation=tf.nn.relu),
                  DenseLayer(out_dim=output_size)])

st = time.perf_counter()
train_model(mlp_model2, train_data, val_data, loss=cross_entropy_loss, acc=accuracy, optimizer=Adam(), epochs=10)
ft = time.perf_counter()
print(f"\nОбучение заняло {ft - st:0.4f} секунд")

Обучение многослойного персептрона из 2х слоев

Эпоха: 0
Обучающая ошибка: 0.224, Обучающая точность: 0.934
Валидационная ошибка: 0.120, Валидационная точность: 0.965
Эпоха: 1
Обучающая ошибка: 0.079, Обучающая точность: 0.977
Валидационная ошибка: 0.089, Валидационная точность: 0.975
Эпоха: 2
Обучающая ошибка: 0.047, Обучающая точность: 0.986
Валидационная ошибка: 0.082, Валидационная точность: 0.977
Эпоха: 3
Обучающая ошибка: 0.032, Обучающая точность: 0.990
Валидационная ошибка: 0.094, Валидационная точность: 0.975
Эпоха: 4
Обучающая ошибка: 0.027, Обучающая точность: 0.992
Валидационная ошибка: 0.095, Валидационная точность: 0.975
Эпоха: 5
Обучающая ошибка: 0.023, Обучающая точность: 0.992
Валидационная ошибка: 0.096, Валидационная точность: 0.976
Эпоха: 6
Обучающая ошибка: 0.019, Обучающая точность: 0.994
Валидационная ошибка: 0.106, Валидационная точность: 0.976
Эпоха: 7
Обучающая ошибка: 0.016, Обучающая точность: 0.995
Валидационная ошибка: 0.099, Валидационная точность: 0.978


Обучение многослойного персептрона из 4х слоев

In [ ]:
print('Обучение многослойного персептрона из 4х слоев\n')
mlp_model4 = MLP([DenseLayer(out_dim=hidden_layer_1_size, activation=tf.nn.relu),
                  DenseLayer(out_dim=hidden_layer_2_size, activation=tf.nn.relu),
                  DenseLayer(out_dim=hidden_layer_3_size, activation=tf.nn.relu),
                  DenseLayer(out_dim=hidden_layer_4_size, activation=tf.nn.relu),
                  DenseLayer(out_dim=output_size)])

st = time.perf_counter()
train_model(mlp_model4, train_data, val_data, loss=cross_entropy_loss, acc=accuracy, optimizer=Adam(), epochs=10)
ft = time.perf_counter()
print(f"\n\nОбучение заняло {ft - st:0.4f} секунд")

Обучение многослойного персептрона из 4х слоев

Эпоха: 0
Обучающая ошибка: 0.237, Обучающая точность: 0.931
Валидационная ошибка: 0.120, Валидационная точность: 0.967
Эпоха: 1
Обучающая ошибка: 0.089, Обучающая точность: 0.973
Валидационная ошибка: 0.112, Валидационная точность: 0.968
Эпоха: 2
Обучающая ошибка: 0.057, Обучающая точность: 0.982
Валидационная ошибка: 0.087, Валидационная точность: 0.976
Эпоха: 3
Обучающая ошибка: 0.044, Обучающая точность: 0.986
Валидационная ошибка: 0.105, Валидационная точность: 0.974
Эпоха: 4
Обучающая ошибка: 0.035, Обучающая точность: 0.989
Валидационная ошибка: 0.103, Валидационная точность: 0.976
Эпоха: 5
Обучающая ошибка: 0.033, Обучающая точность: 0.989
Валидационная ошибка: 0.097, Валидационная точность: 0.976
Эпоха: 6
Обучающая ошибка: 0.028, Обучающая точность: 0.991
Валидационная ошибка: 0.102, Валидационная точность: 0.977
Эпоха: 7
Обучающая ошибка: 0.020, Обучающая точность: 0.993
Валидационная ошибка: 0.121, Валидационная точность: 0.974


Точность на тестовой выборке

In [ ]:
mlp_model_export2 = ExportModule(model=mlp_model2, preprocess=preprocess_test, class_pred=class_pred_test)
mlp_model_export4 = ExportModule(model=mlp_model4, preprocess=preprocess_test, class_pred=class_pred_test)

test_classes2 = mlp_model_export2(x_test)
test_classes4 = mlp_model_export4(x_test)
test_acc2 = accuracy_score(test_classes2, y_test)
test_acc4 = accuracy_score(test_classes4, y_test)

print(f"Точность на тестовой выборке персептрона из 2х слоев: {test_acc2:.3f}")
print(f"Точность на тестовой выборке персептрона из 4х слоев: {test_acc4:.3f}")

Точность на тестовой выборке персептрона из 2х слоев: 0.979
Точность на тестовой выборке персептрона из 4х слоев: 0.979


**Увеличение слоев - увеличивает время обучения, но не всегда улучшает результат, в данном случае у тестовой выборки остался такой же результат.**

#####  **2. В качестве датасета использовать любой из наборов данных TensorFlow. Обучить модель. Добейтесь результата распознования более 85% на тестовой выборке. [Датасеты на выбор](https://www.tensorflow.org/datasets/overview). За задание дополнительно - 3 балла.**

Инициализация функций/классов

In [ ]:
def xavier_init(shape):
    in_dim, out_dim = shape
    xavier_lim = tf.sqrt(6.) / tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
    weight_vals = tf.random.uniform(shape=(in_dim, out_dim), minval=-xavier_lim, maxval=xavier_lim)
    return weight_vals


def cross_entropy_loss(y_pred, y):
    sparse_ce = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_pred)
    return tf.reduce_mean(sparse_ce)


def accuracy(y_pred, y):
    class_preds = tf.argmax(tf.nn.softmax(y_pred), axis=1)
    is_equal = tf.equal(y, class_preds)
    return tf.reduce_mean(tf.cast(is_equal, tf.float32))


class DenseLayer(tf.Module):
    def __init__(self, out_dim, weight_init=xavier_init, activation=tf.identity):
        self.out_dim = out_dim
        self.weight_init = weight_init
        self.activation = activation
        self.built = False

    @tf.function
    def __call__(self, x):
        if not self.built:
            self.in_dim = x.shape[1]
            self.w = tf.Variable(self.weight_init(shape=(self.in_dim, self.out_dim)))
            self.b = tf.Variable(tf.zeros(shape=(self.out_dim,)))
            self.built = True
        y = tf.add(tf.matmul(x, self.w), self.b)
        return self.activation(y)


class MLP(tf.Module):
    def __init__(self, layers):
        self.layers = layers

    @tf.function
    def __call__(self, x, preds=False):
        for layer in self.layers:
            x = layer(x)
        return x


class Adam:
    def __init__(self, learning_rate=1e-3, beta_1=0.9, beta_2=0.999, ep=1e-7):
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.learning_rate = learning_rate
        self.ep = ep
        self.t = 1.
        self.v_dvar, self.s_dvar = [], []
        self.built = False

    def apply_gradients(self, grads, vars):
        if not self.built:
            for var in vars:
                v = tf.Variable(tf.zeros(shape=var.shape))
                s = tf.Variable(tf.zeros(shape=var.shape))
                self.v_dvar.append(v)
                self.s_dvar.append(s)
            self.built = True

        for i, (d_var, var) in enumerate(zip(grads, vars)):
            self.v_dvar[i].assign(self.beta_1 * self.v_dvar[i] + (1 - self.beta_1) * d_var)
            self.s_dvar[i].assign(self.beta_2 * self.s_dvar[i] + (1 - self.beta_2) * tf.square(d_var))
            v_dvar_bc = self.v_dvar[i] / (1 - (self.beta_1 ** self.t))
            s_dvar_bc = self.s_dvar[i] / (1 - (self.beta_2 ** self.t))
            var.assign_sub(self.learning_rate * (v_dvar_bc / (tf.sqrt(s_dvar_bc) + self.ep)))
        self.t += 1.
        return


def train_step(x_batch, y_batch, loss, acc, model, optimizer):
    # Обновляем состояние модели с учетом пакета данных
    with tf.GradientTape() as tape:
        y_pred = model(x_batch)
        batch_loss = loss(y_pred, y_batch)
    batch_acc = acc(y_pred, y_batch)
    grads = tape.gradient(batch_loss, model.variables)
    optimizer.apply_gradients(grads, model.variables)
    return batch_loss, batch_acc


def train_model(mlp, train_data, loss, acc, optimizer, epochs):
    # Отформатируем тренировочный цикл и начнем обучение
    for epoch in range(epochs):
        batch_losses_train, batch_accs_train = [], []
        # Выполняем итерацию по обучающим данным
        for x_batch, y_batch in train_data:
            batch_loss, batch_acc = train_step(x_batch, y_batch, loss, acc, mlp, optimizer)
            batch_losses_train.append(batch_loss)
            batch_accs_train.append(batch_acc)
        # Следим за производительностью модели на уровне эпохи
        train_loss, train_acc = tf.reduce_mean(batch_losses_train), tf.reduce_mean(batch_accs_train)
        print(f"Эпоха: {epoch}")
        print(f"Обучающая ошибка: {train_loss:.3f}, Обучающая точность: {train_acc:.3f}")


def preprocess(x, y):
    x = tf.reshape(x, shape=(-1, 784))
    x = x / 255
    return x, y


def process_test(test_data, model):
    equal_test = []
    for x_batch, y_batch in test_data:
        batch_test = []
        y_pred = model(x_batch)
        for i in tf.nn.softmax(y_pred, axis=1):
            batch_test.append(tf.argmax(i))
        is_equal = tf.equal(batch_test, y_batch)
        equal_test.extend(is_equal)
    return tf.reduce_mean(tf.cast(equal_test, tf.float32))

Загрузка данных.

In [ ]:
train_data, test_data = tfds.load("fashion_mnist", split=['train', 'test'], as_supervised=True, batch_size=128)
train_data, test_data = train_data.map(preprocess), test_data.map(preprocess)

Обучение многослойного персептрона из 2х слоев

In [ ]:
hidden_layer_1_size = 700
hidden_layer_2_size = 500
output_size = 10

mlp_model = MLP([DenseLayer(out_dim=hidden_layer_1_size, activation=tf.nn.relu),
                 DenseLayer(out_dim=hidden_layer_2_size, activation=tf.nn.relu),
                 DenseLayer(out_dim=output_size)])
train_model(mlp_model, train_data, loss=cross_entropy_loss, acc=accuracy, optimizer=Adam(), epochs=10)

Эпоха: 0
Обучающая ошибка: 0.470, Обучающая точность: 0.831
Эпоха: 1
Обучающая ошибка: 0.349, Обучающая точность: 0.871
Эпоха: 2
Обучающая ошибка: 0.312, Обучающая точность: 0.885
Эпоха: 3
Обучающая ошибка: 0.287, Обучающая точность: 0.893
Эпоха: 4
Обучающая ошибка: 0.269, Обучающая точность: 0.899
Эпоха: 5
Обучающая ошибка: 0.249, Обучающая точность: 0.906
Эпоха: 6
Обучающая ошибка: 0.236, Обучающая точность: 0.912
Эпоха: 7
Обучающая ошибка: 0.224, Обучающая точность: 0.916
Эпоха: 8
Обучающая ошибка: 0.212, Обучающая точность: 0.920
Эпоха: 9
Обучающая ошибка: 0.199, Обучающая точность: 0.925


Точность на тестовой выборке

In [ ]:
test_accuracy = process_test(test_data, model=mlp_model)
print(f"Точность на тестовой выборке: {test_accuracy:.3f}")

Точность на тестовой выборке: 0.886
